In [ ]:
import numpy as np

In [ ]:
'''
Defining global variables
'''

# evaluation threshold
beta = 0.2
# consensus threshold
gamma = 0.75 # changed from .75 to test
# deviation threshold
alpha = 0.20
# efficiency threshold
chi = 0.05


In [ ]:
# m - no. of experts
m = 5
# n - no. of alternatives
n = 4
# P - 3d preference relation matrix of dimension m*n*n (FPR)
P = [
    [[0.5, 0.44, 0.40, 0.36],
[0.56, 0.5, 0.79, 0.58],
[0.60, 0.21, 0.5, 0.98],
[0.94, 0.42, 0.02, 0.5]],

[[0.5, 0.22, 0.99, 0.39],
[0.78, 0.5, 0.37, 0.32],
[0.01, 0.63, 0.5, 0.35],
[0.61, 0.68, 0.65, 0.5]],

[[0.5, 0.14, 0.11, 0.01],
[0.86, 0.5, 0.44, 0.05],
[0.89, 0.56, 0.5, 0.06],
[0.99, 0.95, 0.94, 0.5]],

[[0.5, 0.11, 0.62, 0.55],
[0.89, 0.5, 0.82, 0.04],
[0.38, 0.18, 0.5, 0.98],
[0.45, 0.96, 0.02, 0.5]],

[[0.5, 0.97, 0.43, 0.18],
[0.03, 0.5, 0.56, 0.50],
[0.57, 0.44, 0.5, 0.49],
[0.82, 0.50, 0.51, 0.5]]
]

P = np.array(P)
print("P global shape: ", P.shape)
# weights - weight of experts 1*m
weights = [0.3, 0.2, 0.2, 0.1, 0.2]
weights = np.array(weights)
# ST - interaction strength graph (m*m)
# trustMat = np.random.rand(5,5)
# trustMat = np.round(trustMat, 1)
# np.fill_diagonal(trustMat, 0)
# trustMat

trustMat = np.array([[0. , 0.7, 0.2, 0.3, 0.5],
       [0.1, 0. , 0.4, 0.9, 0.7],
       [0.5, 0.4, 0. , 0. , 0.2],
       [0.1, 0.8, 0. , 0. , 0.3],
       [0.7, 0.3, 0.2, 0.4, 0. ]])


# trustMat = [[0, 0.3, 0.4, 0, 0.2], 
# 	 [0.4, 0, 0.1, 0.1, 0.2],
# 	 [0.3, 0.5, 0, 0.1, 0.2],
#    [0, 0, 0, 0, 0],
#    [0.1, 0.1, 0.2, 0.2, 0]
#    ]
# trustMat = np.array(trustMat)

selfTrust = np.random.rand(5)
selfTrust = np.round(selfTrust, 1)

moderatorTrust = np.random.rand(5)
moderatorTrust = np.round(moderatorTrust, 1)

P global shape:  (5, 4, 4)


In [ ]:
def collective_matrix(w, P):  # w is normalized weights after update_MBweights for consensus measuring, and original for finding manipulating experts
  Pc = np.zeros((n,n))
  for k in range(m):
    Pc = Pc + w[k]*P[k]
  return Pc
# print(collective_matrix(weights,P))

In [ ]:
def NC1(k,Pc,P):
  # print("P shape in nc1: ", P.shape)
  temp = np.abs(P[k] - Pc)  #for the case i=j, both Pij and Pcij are 0.5, 
  # so the difference does not contribute to sum, same as in paper
  nc1 = np.sum(temp)
  nc1 = nc1/(n*n - n)
  return nc1

In [ ]:
def NC2(k):
  phi_k = np.sum(P[k],1) # 1D array having sum of preference of each alternative
  # print("phi_k: ", phi_k)
  l = -1
  nc2= -999
  for i in range(n):
    min_val = 999
    for j in range(n):
       if j!=i:   
        diff = phi_k[i]-phi_k[j] 
        min_val = min(min_val, diff)
    if min_val>nc2:
      nc2 = min_val
      l = i
  return nc2,l


In [ ]:
def collectiveNc2(Pc,l):
  # integer having sum of collective preference of lth alternative
  phi_xl = 0
  for i in range(n):
    phi_xl += Pc[l][i]
  phi_xj = np.sum(Pc,1) # 1D array having sum of collective preference of each alternative=
  collective_nc2 = 999
  for i in range(n):
    if phi_xl > phi_xj[i]:
      collective_nc2 = min(collective_nc2, phi_xl-phi_xj[i])
  return collective_nc2

In [ ]:

# Sub-network detection method based on WeChat-like interaction network
# Algorithm 1
def detectSubgroups(trustMat, k=1): #trustMat is the mxm trust matrix
  m = trustMat.shape[0]
	# step 1
  C = np.identity(m) + trustMat
  for i in range(m):
    for j in range(m):
      if C[i][j] > 0:
       C[i][j] = 1
  #print(C)
  #step 2
  Ebar = [] 
  nc = []
  for i in range(m):
   nci = np.count_nonzero(C[:,i])
   #print(trustMat[:,i])
   if nci > 0:
   	Ebar.append((nci, i))
  Ebar.sort(reverse=True)
  # print(Ebar)
  #step 3
  groups = []
  while Ebar:
    ejk = Ebar[0][1]
    ncjk = Ebar[0][0]
    EkLeader = []
    for i in Ebar:
      ncp = i[0]
      ep = i[1]
      if ncjk == ncp and C[ejk][ep]==1:
        EkLeader.append(ep)
    # E bar  = Ebar \ EkLeader
    Ebar = [i for i in Ebar if i[1] not in EkLeader]
    print("Group ", k)
    k+=1
    print("EkLeader:", EkLeader)
    #print(Ebar)
    EkFollower = []
    for i in Ebar:
      ei = i[1]
      for j in EkLeader:
        if C[ei][j] == 1:
          EkFollower.append(ei)
          break
    #print(Ebar)
    # E bar  = Ebar \ EkFollower
    Ebar = [i for i in Ebar if i[1] not in EkFollower]
    print("EkFollower:", EkFollower)
    #print(Ebar)
    EkLeader.sort()
    EkFollower.sort()
    Ek = [EkLeader, EkFollower]
    groups.append(Ek)
  return groups
    

In [ ]:
#Procedure to update weights of manipulators
def sigmah(h,collective_nc2,Pc,P):
  nc2h = NC2(h)[0]
  num = collective_nc2/nc2h
  num = num*(1-NC1(h,Pc,P))
  den = 1+ (1 - (collective_nc2/nc2h))*NC1(h,Pc,P)
  # print("expert: ", h, "nc2h: ", nc2h, "collective: ", collective_nc2)
  # print("num: ", num, "den: ", den )
  return num/den

def update_MBweights(weights, nc2l, Pc, manipulatorIndices,P):
  for i in manipulatorIndices:
    sigma = sigmah(i,nc2l,Pc,P)
    weights[i] = weights[i]*sigma

In [ ]:
# normalize weights of other DM's 
def normalize_weights(MBexperts, weights):
  mb_experts_sum = 0
  for expert in MBexperts:
    mb_experts_sum += weights[expert]
  nfactor = (1-mb_experts_sum)/(m - len(MBexperts))
 

  for i in range(m):
    if i not in MBexperts:
      weights[i] *= nfactor
  # print("in normalize: ", weights, "factor: ", nfactor)
  return weights

In [ ]:
def consensus_degree_expert(k, Pc):
  temp = 1-(np.abs(P[k] - Pc))  #for the case i=j, both Pij and Pcij are 0.5, so the difference does not contribute to sum, same as in paper
  consensus_degree = np.sum(temp)/(n*(n-1))
  return consensus_degree

In [ ]:
def lambda_den(group):
  w_sum = 0
  for l in group:
    for expert in l:
        w_sum+=weights[expert]
  return w_sum
def consensus_degree_groups(groups,Pc):
  ACD = []
  for group in groups:
    w_sum = lambda_den(group)
    cd_group = 0
    for l in group:
      for expert in l:
        lambdak = weights[expert]/w_sum
        cd_group += lambdak*consensus_degree_expert(expert,Pc)
      ACD.append(cd_group)
    return ACD

def consensus_degree_groups(groups,Pc):
  ACD = []
  for group in groups:
    w_sum = lambda_den(group)
    cd_group = 0
    for l in group: # l is for the two lists, leader and follower
      for expert in l:
        lambdak = weights[expert]/w_sum
        cd_group += lambdak*consensus_degree_expert(expert,Pc)
    ACD.append(cd_group)
  return ACD

In [ ]:
# influence matrix matrix
def getInfluenceMatrix(group, trustMat = trustMat, selfTrust = selfTrust, moderatorTrust = moderatorTrust):
  mk = len(group)
  TD = np.zeros((mk, mk+1))
  z = []
  # print(trustMat.shape)
  for i in range(mk):
    for j in range(mk):
      TD[i][j] = trustMat[group[i]][group[j]]
    TD[i][mk] = moderatorTrust[group[i]]
    z.append(selfTrust[group[i]])

  #normalize TD
  sum_of_rows = TD.sum(axis=1)
  TD = TD / sum_of_rows[:, np.newaxis]
  
  # obtain influence matrix W for this group
  q = np.eye(mk) - np.diag(z)
  W = np.c_[np.diag(z), np.zeros(mk)] + np.matmul(q, TD) #equation 14
  return W

In [ ]:
def getNextPreference(group, Pt, Pct):
  # here and in getInfluenceMatrix, group is assumed to be a list of experts, not list of list having leaders and followers
  Wk = getInfluenceMatrix(group)

  # obtain pij, of size (mk+1)*(m)*(m), current preferences of these experts and Pc
  pij = []
  for i in group:
    pij.append(Pt[i])
  pij.append(Pct)
  pij = np.array(pij)

  #evolved preferences
  pNext = []
  mk = len(group)
  for i in range(mk):   #obtain next preference of each expert as per eq 15
    prefI = np.zeros((n,n))
    for j in range(mk+1):
      prefI += Wk[i][j]*pij[j]
    pNext.append(prefI)
  pNext = np.array(pNext) 
  # print(pNext.shape)


  return np.array(pNext)
    
getNextPreference([1,2], P, collective_matrix(weights, P))




array([[[0.5    , 0.2498 , 0.8016 , 0.3298 ],
        [0.7502 , 0.5    , 0.4216 , 0.2994 ],
        [0.1984 , 0.5784 , 0.5    , 0.3654 ],
        [0.6882 , 0.7006 , 0.6346 , 0.5    ]],

       [[0.5    , 0.19235, 0.2987 , 0.10735],
        [0.80765, 0.5    , 0.45245, 0.1358 ],
        [0.7013 , 0.54755, 0.5    , 0.1803 ],
        [0.90615, 0.8642 , 0.8197 , 0.5    ]]])

In [ ]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [ ]:
def identifyNB(flatG,Pc,Pc_prev,P,Pprev):
  NB = []
  for k in flatG:
    nc1 = NC1(k,Pc,P)
    nc1_prev = NC1(k,Pc_prev,Pprev)
    nc3 = (nc1_prev - nc1)/nc1_prev
    if nc1 >= (1-gamma) and nc3 <= chi:
      NB.append(k)
  return NB

In [ ]:
def update_NBweights(weights,NB,Pc,P):
  for ek in NB:
    weights[ek] = weights[ek]*(1-NC1(ek,Pc,P))

In [ ]:
def score(Pc,i):
  sum =0
  for j in range(n):
    sum+=Pc[i][j]
  return sum


In [ ]:
# The main steps of the GDM method
def GDM(P=P, trustMat = trustMat, weights=weights, alpha = alpha, beta = beta, gamma = gamma, chi = chi, t=0):
  # see if the graph shows trust mat, or interaction strength from which trust mat is to be calculated 
  #step 1
  groups = detectSubgroups(trustMat)
  print("Sub-groups in interaction network are: ",groups)
  #step 2
  Pc = collective_matrix(weights, P)
  print("Collective Matrix Pc: ", Pc)
  #experts that greatly differ from other experts
  R1experts = []
  for k in range(m):
    nc1k = NC1(k,Pc,P)
    if nc1k >= alpha:
      R1experts.append(k)
  print("R1 experts: ",R1experts)
  #experts that are inclined towards some alternative l
  R2experts = []
  MB = {}

  for k in range(m):
    nc2k, l = NC2(k)
    # print(k, ":", nc2k)
    collectiveNC2 = collectiveNc2(Pc,l) #generally, we choose beta > nc2l, that has to be implemented
    # print("collectiveNC2:", collectiveNC2)
    if nc2k >= beta:
      R2experts.append((l,k))
      if k in R1experts:
        if l not in MB: MB[l] = [k]
        else: MB[l].append(k)
  print("R2 experts: ", R2experts)
  # print("experts following R1 and R2 supporting some  alternative: ", MB)
  #R3
  groups = detectSubgroups(trustMat)
  nGrps = len(groups)
  MBexperts = {}  # dict having the alternative and corresponding manipulative experts

  MBexpertsList = [] # list having all MBexperts
  
  for i in MB:
    MBi = MB[i]
    for ek in groups:
      MBik = [j for j in MB[i] if j in ek[0] or j in ek[1]] # eq 6 - members in group Ek who follow R1 and R2 for alternative xi
      if set(MBik) & set(ek[0]):  
        # MBexperts += MBik #eq 7 - If these have an opinion leader, we say that these experts are involved in manipulative behaviour
        if i not in MBexperts:
          MBexperts[i] = MBik
        else: MBexperts[i].exted(MBik)
        MBexpertsList += MBik
  print("Manipulating experts for alternatives: ", MBexperts)

  '''Step 3: Update weights of Manipulating DMs, normalize weights of other DMs, 
  remove manipulators and obtain updated subgroups
  ''' 

  MBorNBexists = True

  #To get back to step 3, i.e keep updating weights until manipulative or non-coopertaive behaviour exists
  while MBorNBexists:

    # print(weights)
    # Update weights
    for alternative in MBexperts:
      update_MBweights(weights, collectiveNc2(Pc, alternative), Pc, MBexperts[alternative],P)
    # print(weights)
    weights = normalize_weights(MBexpertsList, weights)
    # print("normalized weights", weights)
    # remove manipulative experts from subgroups
    for group in groups:
      group[0]  = [i for i in group[0] if i not in MBexpertsList]
      group[1] = [i for i in group[1] if i not in MBexpertsList]
    # print(MBexpertsList)
    # print(groups)

    '''Step 4: Obtain updated collective matrix using equation 8, and consensus degree of each updated subgroup
    '''
    Pc = collective_matrix(weights, P)

    # GLT - groups with consensus degree lower than gamma
    ACD = consensus_degree_groups(groups,Pc)
    # print("Aggregated consensus degree: ", ACD)
    GLT = []
    for i in range(len(groups)):
      if(ACD[i]<gamma):
        GLT.append(i)
    # print("GLT: ",GLT)
    # print("ACD: ", ACD)

    GLT = [groups[0]]
    # print("groups: ", groups)

    '''Step 5: CRP for groups with consensus degree less than gamma
    ''' 
    gamma = .95 #to test
    for g in GLT:
      # for subgroups with memeber in GLT (non outliers)
      # print(g)
      # print("flat", flatten(g))
      cdg = consensus_degree_groups([g], Pc)
      # print("gamma: ",gamma, "CD: ", cdg)
      # if the consensus degree of this group has not reached threshold
      r=0
      while cdg[0] < gamma:
        # print(cdg[0])
      # for h in range(1):
        r = r + 1
        print("consensus degree after round", r,": ", cdg)
        # obtain next evolution based on feedback
        flatG = flatten(g)
        Pnext = getNextPreference(flatG, P, Pc)

        P_prev = P
        # update P as per Pnext
        for i in range(len(flatG)):
          P[flatG[i]] = Pnext[i]
        # update Pc 
        Pc_prev = Pc
        Pc = collective_matrix(weights, P)
        cdg = consensus_degree_groups([g], Pc)
        # print(cdg)

        ''' Step 6: Identify non cooperative behaviour
        '''
        NB = identifyNB(flatG,Pc,Pc_prev,P,P_prev)
        if r == 1:
          print("Experts with Non-Cooperative behaviour: ", NB)
        
        '''Step 7: If there are DMs with NC Behaviour, penalize weights using Algo 3 and repeat
        '''
        #complete this
        # influenceMat =  getInfluenceMatrix(NB)  #NB contains non coopeartive beharioral experts in this group
     
        update_NBweights(weights,NB,Pc,P)
        # if NC behavior exists, make it true so that the loop from step 3 continues
        if NB: MBorNBexists = False

  print("Final consensus degree: ", cdg)
  
  '''Step 9: Selection
  '''
  # selected = 0
  # max_score = -999
  # for i in range(n):
  #   score_i = score(Pc,i)
  #   print(i, ":", score_i)
  #   if(score_i>max_score):
  #     max_score = score_i
  #     selected = i

  scores = [score(Pc, i) for i in range(n)]
  print("Scores of alternatives are: ", scores)
  print("Selected: ", scores.index(max(scores)))

    

  # if MBexperts:
  #   #step 3
  #   update_MBweights(weights, collectiveNC2(, Pc, MBexperts)
  #   print(weights)
GDM()

Group  1
EkLeader: [4, 1, 0]
EkFollower: [3, 2]
Sub-groups in interaction network are:  [[[0, 1, 4], [2, 3]]]
Collective Matrix Pc:  [[0.5   0.409 0.488 0.279]
 [0.591 0.5   0.593 0.352]
 [0.512 0.407 0.5   0.572]
 [0.811 0.648 0.428 0.5  ]]
R1 experts:  [1, 2, 3]
R2 experts:  [(3, 1), (3, 2), (1, 3), (3, 4)]
Group  1
EkLeader: [4, 1, 0]
EkFollower: [3, 2]
Manipulating experts for alternatives:  {3: [1, 2]}
consensus degree after round 1 :  [0.8618918490830527]
Experts with Non-Cooperative behaviour:  [0, 4, 3]
consensus degree after round 2 :  [0.9048150063446329]
consensus degree after round 3 :  [0.901638677272024]
consensus degree after round 4 :  [0.9175960730312542]
consensus degree after round 5 :  [0.9424930985576905]
Final consensus degree:  [0.9703584666040109]
Scores of alternatives are:  [0.3886659705628978, 0.40029633583256896, 0.32816057131411014, 0.523132495608055]
Selected:  3
